# intermediate_SARC
This notebook takes our custom XED binary datas_et and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [1]:
import os
os.chdir('..')
# os.getcwd( )

In [2]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [3]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_SARC_01"
params.dataset_path = "data/inter_SARC/itesd_sarc_balanced.hf"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### SARC

In [4]:
datasets = load_from_disk(params.dataset_path)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment', 'num_word_text'],
        num_rows: 80000
    })
    validation: Dataset({
        features: ['label', 'text', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment', 'num_word_text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['label', 'text', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment', 'num_word_text'],
        num_rows: 10000
    })
})

In [5]:
# we will need to view and prep the datasets
# this is more easily done as dataframes
train_df = datasets['train'].to_pandas()
validate_df = datasets['validation'].to_pandas()

In [6]:
# view training dataset
print("train_df Info:")
print(train_df.info())
print("\ntrain_df Value Counts")
print(train_df['label'].value_counts())

train_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           80000 non-null  int64 
 1   text            80000 non-null  object
 2   author          80000 non-null  object
 3   subreddit       80000 non-null  object
 4   score           80000 non-null  int64 
 5   ups             80000 non-null  int64 
 6   downs           80000 non-null  int64 
 7   date            80000 non-null  object
 8   created_utc     80000 non-null  object
 9   parent_comment  80000 non-null  object
 10  num_word_text   80000 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 6.7+ MB
None

train_df Value Counts
0    40000
1    40000
Name: label, dtype: int64


In [7]:
# view validation dataset
print("validate_df Info:")
print(validate_df.info())
print("\n validate_df Value Counts")
print(validate_df['label'].value_counts())

validate_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           10000 non-null  int64 
 1   text            10000 non-null  object
 2   author          10000 non-null  object
 3   subreddit       10000 non-null  object
 4   score           10000 non-null  int64 
 5   ups             10000 non-null  int64 
 6   downs           10000 non-null  int64 
 7   date            10000 non-null  object
 8   created_utc     10000 non-null  object
 9   parent_comment  10000 non-null  object
 10  num_word_text   10000 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 859.5+ KB
None

 validate_df Value Counts
1    5006
0    4994
Name: label, dtype: int64


## Preprocess

In [8]:
train_token_ids, train_attention_masks = encode_text(train_df.text.values)
validate_token_ids, validate_attention_masks = encode_text(validate_df.text.values)

In [9]:
train_features = []
for i in range(len(train_token_ids)):
    train_features.append({'label': train_df.label.values[i], 
                           'input_ids': train_token_ids[i], 
                           'attention_mask':train_attention_masks[i]})

validate_features = []
for i in range(len(validate_token_ids)):
    validate_features.append({'label': validate_df.label.values[i], 
                              'input_ids': validate_token_ids[i],
                              'attention_mask':validate_attention_masks[i]})

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [10]:
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [11]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [12]:
model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

output_parameters()

Device: mps

          Training Dataset: data/inter_SARC/itesd_sarc_balanced.hf
          Number of Labels: 2
          Batch Size: 16
          Learning Rate: 1e-05
          Weight Decay: 0
          Epochs: 10
          Output Directory: model_saves/intermediate_SARC_01
          Save Frequency: 1
          Checkpoint Frequency: 1
          Max Length: 256
          


Fit the model to our training data.

In [13]:
trainer.fit()

	 Validation 624: 100%|██████████| 625/625 [00:52<00:00, 11.82batch/s]

 	 - Train loss: 0.553480
	 - Validation Loss: 0.507682
	 - Validation Accuracy: 0.752300
	 - Validation F1: 0.727120
	 - Validation Recall: 0.707129
	 - Validation Precision: 0.776271 

	 * Model @ epoch 1 saved to model_saves/intermediate_SARC_01/E01_A0.75_F0.73
	 * Model checkpoint saved to model_saves/intermediate_SARC_01/E01_A0.75_F0.73/checkpoint.pt

	 Validation 624: 100%|██████████| 625/625 [00:50<00:00, 12.46batch/s]

 	 - Train loss: 0.480337
	 - Validation Loss: 0.497933
	 - Validation Accuracy: 0.756200
	 - Validation F1: 0.748114
	 - Validation Recall: 0.769578
	 - Validation Precision: 0.751305 

	 * Model @ epoch 2 saved to model_saves/intermediate_SARC_01/E02_A0.76_F0.75
	 * Model checkpoint saved to model_saves/intermediate_SARC_01/E02_A0.76_F0.75/checkpoint.pt

	 Validation 624: 100%|██████████| 625/625 [00:50<00:00, 12.30batch/s]

 	 - Train loss: 0.417137
	 - Validation Loss: 0.524495
	 - Valida